In [1]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import requests
from bs4 import BeautifulSoup
from html_table_parser import parser_functions
import pandas as pd

/Users/leejimin/Documents/dev/개연_졸업사정/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [16]:
# 드라이버 초기화
driver = webdriver.Chrome()

url = "https://ai.dongguk.edu/page/30427"

In [17]:
try:
    driver.get(url)
    element = WebDriverWait(driver, 10) # request wait 걸기
    html = driver.page_source # page_source 얻기
    soup = BeautifulSoup(html, 'html.parser') # get html

    table = soup.select_one('.tbl') 
    # 테이블 데이터 추출
    def parse_table(table):
        rows = table.find_all('tr')
        data = []
        rowspan_map = {}  # rowspan을 처리하기 위한 맵

        for row_idx, row in enumerate(rows):
            row_data = []
            cells = row.find_all(['td', 'th'])
            col_idx = 0

            for cell in cells:
                while col_idx in rowspan_map:
                    # 이미 rowspan이 적용된 셀이 있는지 확인
                    row_data.append(rowspan_map[col_idx]['text'])
                    rowspan_map[col_idx]['count'] -= 1
                    if rowspan_map[col_idx]['count'] == 0:
                        del rowspan_map[col_idx]
                    col_idx += 1

                cell_text = cell.get_text(strip=True)
                rowspan = int(cell.get('rowspan', 1))
                colspan = int(cell.get('colspan', 1))

                for i in range(colspan):
                    row_data.append(cell_text)

                if rowspan > 1:
                    for i in range(colspan):
                        span_col_idx = col_idx + i
                        if span_col_idx not in rowspan_map:
                            rowspan_map[span_col_idx] = {'text': cell_text, 'count': rowspan - 1}
                        else:
                            rowspan_map[span_col_idx]['count'] += (rowspan - 1)

                col_idx += colspan

            while col_idx in rowspan_map:
                row_data.append(rowspan_map[col_idx]['text'])
                rowspan_map[col_idx]['count'] -= 1
                if rowspan_map[col_idx]['count'] == 0:
                    del rowspan_map[col_idx]
                col_idx += 1

            data.append(row_data)

        # 데이터 평탄화
        max_length = max(len(row) for row in data)
        flattened_data = [row + [''] * (max_length - len(row)) for row in data]

        return flattened_data

    table_data = parse_table(table)

    # Pandas DataFrame으로 변환
    df = pd.DataFrame(table_data)

    # DataFrame 출력
    df
    
finally:
    driver.quit()